# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import sys
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
# these are for SVD/LSA
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

[nltk_data] Downloading package punkt to /home/umahp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/umahp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/umahp/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/umahp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///data/DisResp.db')

df =pd.read_sql_table('messages', engine)
X = df['message'].values
display(df.info())
display(X.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25865 entries, 0 to 25864
Data columns (total 39 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      25865 non-null  int64 
 1   message                 25865 non-null  object
 2   original                9934 non-null   object
 3   genre                   25865 non-null  object
 4   related                 25865 non-null  int64 
 5   request                 25865 non-null  int64 
 6   offer                   25865 non-null  int64 
 7   aid_related             25865 non-null  int64 
 8   medical_help            25865 non-null  int64 
 9   medical_products        25865 non-null  int64 
 10  search_and_rescue       25865 non-null  int64 
 11  security                25865 non-null  int64 
 12  military                25865 non-null  int64 
 13  water                   25865 non-null  int64 
 14  food                    25865 non-null  int64 
 15  sh

None

(25865,)

In [3]:
Y = df.iloc[:, 4:]
display(Y.shape)
display(Y.head())
# this is to see which categories have few messages associated with them (may be hard to classify)
display(Y.mean(axis=0))
labels = Y.columns.to_list()
display(labels)

(25865, 35)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


related                   0.764624
request                   0.171313
offer                     0.004562
aid_related               0.417862
medical_help              0.080379
medical_products          0.050648
search_and_rescue         0.027837
security                  0.018094
military                  0.033095
water                     0.064411
food                      0.112623
shelter                   0.089039
clothing                  0.015581
money                     0.023236
missing_people            0.011328
refugees                  0.033752
death                     0.046008
other_aid                 0.132380
infrastructure_related    0.065765
transport                 0.046279
buildings                 0.051266
electricity               0.020491
tools                     0.006147
hospitals                 0.010941
shops                     0.004639
aid_centers               0.011947
other_infrastructure      0.044346
weather_related           0.280650
floods              

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [4]:
Y = df.iloc[:, 4:].values
Y.shape

(25865, 35)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words('english')]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
def build_pipeline(clf, svd=False):
    if svd:
    # add on the steps to do LSA
        pipeline = Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('svd', TruncatedSVD(100)),
                    ('nml', Normalizer(copy=False)),       
                    ('multi_clf', MultiOutputClassifier(clf))
        ])
    else:
        pipeline = Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('multi_clf', MultiOutputClassifier(clf))
        ])
    
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

This will compare two solvers on the basic parameters without the additional SVD/LSA feature. I'm doing this preliminary screening because trying to do it all with GridSearchCV is taking way too long.

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
all_results = []

# first try the two solvers with basic params and no SVD
# the SGD parameters are taken from an sklearn example on text classification. will tune later if it comes out 
# the better model of the two.
clfs = [RandomForestClassifier(random_state=42), SGDClassifier(loss='hinge', penalty='l2',\
        alpha=1e-3, random_state=42, max_iter=5, tol=None)]
for clf in clfs:
    model = build_pipeline(clf)
    model.fit(X_train, Y_train) 
    Y_pred = model.predict(X_test)
    cl_name = str(type(clf)).split(".")[-1][:-2]   # thanks stack overflow
    
    get_results(model, Y_test, Y_pred, labels, cl_name, all_results)

/home/umahp/data_science/dsenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umahp/data_science/dsenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umahp/data_science/dsenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/u

/home/umahp/data_science/dsenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umahp/data_science/dsenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umahp/data_science/dsenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/u

In [16]:
result_df = pd.DataFrame(all_results, columns=['classifier', 'category', 'precis_0', 'rcl_0', 'f1_0', 'support_0',\
                                               'precis_1', 'rcl_1', 'f1_1', 'support_1','accuracy','ma_precision',\
                                                'ma_recall', 'ma_f1'])
display(result_df.head())

,classifier,category,precis_0,rcl_0,f1_0,support_0,precis_1,rcl_1,f1_1,support_1,accuracy,ma_precision,ma_recall,ma_f1
0,RandomForestClassifier,related,0.696203,0.401993,0.509688,1505,0.839228,0.946796,0.889773,4962,0.820009,0.767715,0.674395,0.699731
1,RandomForestClassifier,request,0.901577,0.977089,0.937815,5325,0.824713,0.502627,0.624592,1142,0.893304,0.863145,0.739858,0.781204
2,RandomForestClassifier,offer,0.994897,1.000000,0.997442,6434,0.000000,0.000000,0.000000,33,0.994897,0.497449,0.500000,0.498721
3,RandomForestClassifier,aid_related,0.790192,0.838294,0.813533,3729,0.759857,0.696859,0.726996,2738,0.778413,0.775024,0.767577,0.770264
4,RandomForestClassifier,medical_help,0.925655,0.997146,0.960071,5956,0.666667,0.066536,0.120996,511,0.923612,0.796161,0.531841,0.540534


In [17]:
result_df.groupby(['classifier'])[['accuracy', 'ma_precision', 'ma_recall', 'ma_f1']].mean()

,accuracy,ma_precision,ma_recall,ma_f1
classifier,,,,
RandomForestClassifier,0.947001,0.796931,0.595265,0.614294
SGDClassifier,0.936575,0.599069,0.538894,0.540148


In [18]:
result_df.groupby(['classifier'])[['accuracy', 'ma_precision', 'ma_recall', 'ma_f1']].min()

,accuracy,ma_precision,ma_recall,ma_f1
classifier,,,,
RandomForestClassifier,0.778413,0.477336,0.499757,0.484047
SGDClassifier,0.719190,0.435132,0.500000,0.465316


In [19]:
# see how many categories have no messages predicted as belonging to them
result_df.query('(f1_1 ==0)').groupby(['classifier'])['category'].count()

classifier
RandomForestClassifier     6
SGDClassifier             24
Name: category, dtype: int64

Now compare the two solvers on the basic parameters with SVD/LSA.

In [9]:
all_results = []

# the SGD parameters are taken from an sklearn example on text classification. will tune later if it comes out 
# the better model of the two.
clfs = [RandomForestClassifier(random_state=42), SGDClassifier(loss='hinge', penalty='l2',\
        alpha=1e-3, random_state=42, max_iter=5, tol=None)]
for clf in clfs:
    model = build_pipeline(clf, svd=True)
    model.fit(X_train, Y_train) 
    Y_pred = model.predict(X_test)
    cl_name = str(type(clf)).split(".")[-1][:-2]   # thanks stack overflow
    
    get_results(model, Y_test, Y_pred, labels, cl_name, all_results)

/home/umahp/data_science/dsenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umahp/data_science/dsenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umahp/data_science/dsenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/u

/home/umahp/data_science/dsenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umahp/data_science/dsenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umahp/data_science/dsenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/u

/home/umahp/data_science/dsenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umahp/data_science/dsenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umahp/data_science/dsenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/u

In [11]:
# see how much the SVD contributes
display(model.named_steps.svd.explained_variance_ratio_.sum())

0.17054674887381294

In [12]:
result_svd_df = pd.DataFrame(all_results, columns=['classifier', 'category', 'precis_0', 'rcl_0', 'f1_0', 'support_0',\
                                               'precis_1', 'rcl_1', 'f1_1', 'support_1','accuracy','ma_precision',\
                                                'ma_recall', 'ma_f1'])

In [13]:
result_svd_df.groupby(['classifier'])[['accuracy', 'ma_precision', 'ma_recall', 'ma_f1']].mean()

,accuracy,ma_precision,ma_recall,ma_f1
classifier,,,,
RandomForestClassifier,0.941156,0.703638,0.572578,0.586820
SGDClassifier,0.940882,0.595209,0.565326,0.569806


In [14]:
result_svd_df.groupby(['classifier'])[['accuracy', 'ma_precision', 'ma_recall', 'ma_f1']].min()

,accuracy,ma_precision,ma_recall,ma_f1
classifier,,,,
RandomForestClassifier,0.744240,0.466816,0.499752,0.482723
SGDClassifier,0.746405,0.436215,0.500000,0.465934


In [15]:
# see how many categories have no messages predicted as belonging to them
result_svd_df.query('(f1_1 ==0)').groupby(['classifier'])['category'].count()

classifier
RandomForestClassifier    11
SGDClassifier             24
Name: category, dtype: int64

These results are slightly worse than for not using SVD on these data. Will do the grid search without SVD.

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

The results are displayed above and below.

In [7]:
def get_results(model, y_test, y_pred, labels, cl_name, all_results):
    for i, label in enumerate(labels):
        result = classification_report(y_test[:,i], y_pred[:,i], output_dict=True)
        all_results.append([cl_name, label, result['0']['precision'], result['0']['recall'], \
                            result['0']['f1-score'], result['0']['support'], result['1']['precision'], \
                            result['1']['recall'], result['1']['f1-score'], result['1']['support'],\
                            result['accuracy'], result['macro avg']['precision'],\
                            result['macro avg']['recall'],result['macro avg']['f1-score']])
    return


### 6. Improve your model
Use grid search to find better parameters. 

Code that will run this will multiple solvers and options is below. However, that code runs for an excessive amount of time (>48h and counting). I'm using the preliminary results above to limit the grid search to parameters for Random Forest only, with no SVD.

In [ ]:
# Starting with just Random Forest as way overshot on complexity using 3 solvers and is running forever
# have to limit features here as this also takes several hours to run
# just going to do 1 feature for the vectorizer and one for the classifier.
# when this was run with the commented lines left in, it kept going for 3 days and froze.
# going to also cut cv to 2 to cut down time per iteration.
# this combination actually ran extremely quickly: about an hour (!)
parameters = [
    {                                
#                'vect__ngram_range': [(1,1),(1,2)],
                'tfidf__use_idf': [True, False],
                'multi_clf__estimator__n_estimators':  [100, 200],
#                'multi_clf__estimator__max_features': [0.5, "sqrt"]
    }]

    
# create grid search object
clf = RandomForestClassifier(random_state=42)
pipeline = build_pipeline(clf)
# the multithreading option doesn't seem to work in iPython according to what I can find (failed for me)
cv = GridSearchCV(pipeline, param_grid=parameters, cv=2)
cv.fit(X_train, Y_train)
Y_pred = cv.predict(X_test)

cv_results = []
cl_name = str(type(clf)).split(".")[-1][:-2]   # thanks stack overflow    
get_results(cv, Y_test, Y_pred, labels, cl_name, cv_results)

In [17]:
print("\nBest Parameters:", cv.best_params_)


Best Parameters: {'multi_clf__estimator__n_estimators': 200, 'tfidf__use_idf': True}


In [18]:
cv_df = pd.DataFrame(cv_results, columns=['classifier', 'category', 'precis_0', 'rcl_0', 'f1_0', 'support_0',\
                                               'precis_1', 'rcl_1', 'f1_1', 'support_1','accuracy','ma_precision',\
                                                'ma_recall', 'ma_f1'])

In [19]:
cv_df.groupby(['classifier'])[['accuracy', 'ma_precision', 'ma_recall', 'ma_f1']].mean()

,accuracy,ma_precision,ma_recall,ma_f1
classifier,,,,
RandomForestClassifier,0.946688,0.783446,0.592819,0.611153


Those results didn't improve on the baseline model. But at least it ran in a reasonable amount of time. Will try the other two parameter combinations.

In [21]:
# try the other two parameter combos to see if they can improve the fit
parameters = [
    {                                
                'vect__ngram_range': [(1,1),(1,2)],
#                'tfidf__use_idf': [True, False],
#                'multi_clf__estimator__n_estimators':  [100, 200],
                'multi_clf__estimator__max_features': [0.5, "sqrt"]
    }]

    
# create grid search object
clf = RandomForestClassifier(random_state=42)
pipeline = build_pipeline(clf)
# the multithreading option doesn't seem to work in iPython according to what I can find (failed for me)
cv2 = GridSearchCV(pipeline, param_grid=parameters, cv=2)
cv2.fit(X_train, Y_train)
Y_pred = cv2.predict(X_test)

cv2_results = []
cl_name = str(type(clf)).split(".")[-1][:-2]   # thanks stack overflow    
get_results(cv2, Y_test, Y_pred, labels, cl_name, cv2_results)

/home/umahp/data_science/dsenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umahp/data_science/dsenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umahp/data_science/dsenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/u

In [22]:
print("\nBest Parameters:", cv2.best_params_)


Best Parameters: {'multi_clf__estimator__max_features': 0.5, 'vect__ngram_range': (1, 2)}


In [23]:
cv2_df = pd.DataFrame(cv2_results, columns=['classifier', 'category', 'precis_0', 'rcl_0', 'f1_0', 'support_0',\
                                               'precis_1', 'rcl_1', 'f1_1', 'support_1','accuracy','ma_precision',\
                                                'ma_recall', 'ma_f1'])

Steps 7 & 8 have already been done above. Skipping to pickling.

### 9. Export your model as a pickle file

In [26]:
import pickle
with open('disaster_explore.pkl', 'wb') as outfile:
    pickle.dump(cv, outfile)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
cv2_df.groupby(['classifier'])[['accuracy', 'ma_precision', 'ma_recall', 'ma_f1']].mean()

,accuracy,ma_precision,ma_recall,ma_f1
classifier,,,,
RandomForestClassifier,0.948349,0.766316,0.684536,0.707678


see above

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

This was done above. I tried two solvers (Random Forest and SGD SVM) and tried adding SVD for feature reduction/optimization.
Trying to do a grid search with multiple solvers and options runs for days so I limited the final grid search to the Random Forest without SVD.

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.